In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib as mpl
import cv2
import time
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.initializers import glorot_normal

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image2(img):
    # Apply median filter for noise removal
    img_median = cv2.medianBlur(img, 3)

    # Convert the image to grayscale for OTSU thresholding
    img_gray = cv2.cvtColor(img_median, cv2.COLOR_BGR2GRAY)

    # Apply OTSU thresholding for binarization
    _, binary_image = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Convert binary image to 3 channels
    binary_image_3_channels = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)

    # Display the original, median-filtered, and binarized images for comparison
    # plt.figure(figsize=(10, 5))
    # plt.subplot(131), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
    # plt.subplot(132), plt.imshow(cv2.cvtColor(img_median, cv2.COLOR_BGR2RGB)), plt.title('Median Filtered')
    # plt.subplot(133), plt.imshow(binary_image_3_channels), plt.title('Binarized (3 channels)')
    # plt.show()

    return binary_image_3_channels



In [6]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load the updated Excel file
updated_excel_file = 'updated_excel_file.xlsx'
data = pd.read_excel(updated_excel_file, usecols=range(5))  # Load only the first 5 columns
width = height = 800
batch_size = 32


def load_images_and_labels(file_names_column, age_column, start_idx, end_idx):
    images_batch = []
    labels_batch = []
    
    for index in range(start_idx, end_idx):
        row = data.iloc[index]
        file_name = row[file_names_column]
        age = row[age_column]

        if pd.notnull(age) and pd.notnull(file_name):
            image = cv2.imread(os.path.join('./images', file_name))
            image = preprocess_image2(image)
            image = cv2.resize(image, (width, height))
            images_batch.append(image)
            labels_batch.append(age)

    labels_batch = to_categorical(labels_batch, num_classes=2)
    images_batch = np.array(images_batch) / 255.0
    
    return images_batch, labels_batch

# Load all images and labels
all_images = []
all_labels = []

for start_idx in range(0, len(data), batch_size):
    end_idx = min(start_idx + batch_size, len(data))
    images_batch, labels_batch = load_images_and_labels('File Name', 'Gender', start_idx, end_idx)
    
    all_images.append(images_batch)
    all_labels.append(labels_batch)

# Concatenate the batches to create the final arrays
all_images = np.concatenate(all_images)
all_labels = np.concatenate(all_labels)

# Now, 'all_images' contains all the images and 'all_labels' contains their corresponding labels

# Example usage
print(all_images.shape)  # Print the shape of the array containing all images
print(all_labels.shape)  # Print the shape of the array containing all labels


MemoryError: Unable to allocate 27.5 GiB for an array with shape (1922, 800, 800, 3) and data type float64

In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50,MobileNet
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model


In [ ]:
# This function will create the actual and predicted
# vectors for further evaluation.

def generate_actual_predicted(model, X_test, Y_test):
    # Get the predictions
    Y_pred = model.predict(X_test)

    # Create a list of predictions

    #Converting predictions to label
    predicted = list()
    for i in range(len(Y_pred)):
        predicted.append(np.argmax(Y_pred[i]))

    #Converting one hot encoded test label back to label
    actual = list()
    for i in range(len(Y_test)):
        actual.append(np.argmax(Y_test[i]))

    return actual, predicted


In [ ]:
from keras.callbacks import ModelCheckpoint

# save the best model 
filepath='best_modelNew'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
# This will only use the training data 
# and test it on each partition
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


n_split=5

# We will split the the data n_split times
# and create and for the model with the said
# parameters
images1=all_images[:1000]
labels1=all_labels[:1000]
results = list()
for train_index,test_index in KFold(n_split).split(images1): 
  # use the index to generate training an testing sets
  x_train,x_test=images1[train_index],images1[test_index]
  y_train,y_test=labels1[train_index],labels1[test_index]
  
  # create and fit the model
#   model=build_model1()
          # Input tensor
  #   width=height=1000
  base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(width, height, 3), pooling='avg',
                          alpha=1.0, depth_multiplier=1, dropout=.2)
  base_model.trainable = False
  x_top = base_model.output
  x_out = Dense(128, activation='relu')(x_top)
  out=Dense(2, activation='softmax')(x_out)

  model = Model(base_model.input, out)
  model.summary()

  model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

  #model1.summary()

  # Train the model
  history =model.fit(x_train, y_train,epochs=10, batch_size = 32,callbacks = callbacks_list, validation_split=0.2)



  #history =bilinear_model.fit(x_train, y_train,epochs=20, batch_size = 32,callbacks = callbacks_list, validation_split=0.2)
  
  # print and show generic metrics available
  scores = model.evaluate(x_test,y_test)

  # print the model metrics
  for i,names in zip(np.arange(0,len(model.metrics_names)),model.metrics_names):
      print(model.metrics_names[i],'=',scores[i])
  # calculate and print more metrics
  actual, predicted = generate_actual_predicted(model, x_test, y_test)
  print("precision ",precision_score(actual, predicted, average='macro'))
  print("recall ", recall_score(actual, predicted, average='macro'))
  print("F1-Score ", f1_score(actual, predicted, average='macro'))
  results.append([precision_score(actual, predicted, average='macro'),
                  recall_score(actual, predicted, average='macro'), 
                  f1_score(actual, predicted, average='macro') ])
  
  
  #del bilinear_model